# Split 1

In [311]:
import pandas as pd

#import train and test CSV files
train = pd.read_csv("../5-fold_new/DATA/split1/train/train_split1.csv")
test = pd.read_csv("../5-fold_new/DATA/test.csv")

#take a look at the training data
train.describe(include="all")

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,3.640000e+02,364,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,...,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000
unique,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,3.651918e+07,NaN,14.012467,19.443132,91.008654,645.112912,0.094224,0.097667,0.079566,0.044374,...,15.975813,25.659341,105.173269,851.392308,0.129065,0.234064,0.247093,0.106476,0.280414,0.081702
std,1.411919e+08,NaN,3.524482,4.563821,24.228887,356.700236,0.013602,0.048026,0.074338,0.037194,...,4.824430,6.306664,33.331071,579.786496,0.021498,0.136328,0.193067,0.062580,0.049940,0.015429
min,8.913000e+03,NaN,7.691000,10.720000,47.920000,170.400000,0.052630,0.026500,0.000000,0.000000,...,8.678000,12.870000,54.490000,223.600000,0.084090,0.043270,0.000000,0.000000,0.156600,0.055210
25%,8.986778e+05,NaN,11.600000,16.177500,74.237500,409.025000,0.084430,0.060612,0.026013,0.018793,...,12.785000,20.975000,83.112500,494.500000,0.113675,0.138625,0.106575,0.061995,0.246950,0.070595
50%,9.151645e+05,NaN,13.165000,18.760000,84.715000,537.300000,0.093510,0.085555,0.052950,0.029380,...,14.475000,25.090000,95.120000,642.550000,0.128200,0.200950,0.192050,0.091400,0.275300,0.078270
75%,9.010399e+06,NaN,15.492500,21.850000,102.825000,744.750000,0.103125,0.122625,0.112700,0.063168,...,17.535000,29.350000,117.325000,934.450000,0.141725,0.309025,0.354675,0.145600,0.307550,0.089525


In [312]:
from sklearn.model_selection import train_test_split

x = train.drop(['diagnosis', 'id'], axis=1)
y = train["diagnosis"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [313]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [314]:
# Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

sgd = SGDClassifier()
sgd.fit(x_train, y_train)
y_pred = sgd.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

sgd_acc = accuracy_score(y_test, y_pred)
print("Accuracy of SGD model is: ", sgd_acc)

Confusion Matrix: 
 [[49  1]
 [ 2 21]]


              precision    recall  f1-score   support

           B       0.96      0.98      0.97        50
           M       0.95      0.91      0.93        23

    accuracy                           0.96        73
   macro avg       0.96      0.95      0.95        73
weighted avg       0.96      0.96      0.96        73

Accuracy of SGD model is:  0.958904109589041


# Split 2

In [315]:
import pandas as pd

#import train and test CSV files
train = pd.read_csv("../5-fold_new/DATA/split2/train/train_split2.csv")
test = pd.read_csv("../5-fold_new/DATA/test.csv")

#take a look at the training data
train.describe(include="all")

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,3.640000e+02,364,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,...,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000
unique,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,3.254930e+07,NaN,13.871049,18.989890,90.096758,631.559890,0.095246,0.098559,0.080578,0.044665,...,15.796868,25.192363,103.916016,830.368681,0.129812,0.234327,0.247629,0.105611,0.281948,0.082100
std,1.407212e+08,NaN,3.502110,4.223905,24.129511,350.388382,0.014182,0.048844,0.078082,0.038462,...,4.678765,5.952995,32.474792,552.599552,0.022263,0.139817,0.196572,0.063465,0.056287,0.015859
min,8.670000e+03,NaN,7.691000,9.710000,47.920000,170.400000,0.052630,0.019380,0.000000,0.000000,...,8.678000,12.020000,54.490000,223.600000,0.071170,0.027290,0.000000,0.000000,0.156600,0.055210
25%,8.778670e+05,NaN,11.592500,15.970000,74.222500,408.650000,0.085038,0.061398,0.026623,0.018793,...,12.777500,20.735000,82.755000,494.500000,0.114200,0.137300,0.103950,0.060362,0.245725,0.070830
50%,9.084790e+05,NaN,12.995000,18.585000,83.900000,517.900000,0.094400,0.085745,0.051710,0.029470,...,14.405000,25.035000,94.090000,633.600000,0.129550,0.203400,0.191550,0.091000,0.274100,0.078605
75%,8.711206e+06,NaN,15.197500,21.540000,100.600000,716.675000,0.104450,0.122625,0.111700,0.061593,...,17.505000,28.872500,116.750000,933.475000,0.142525,0.305725,0.358225,0.145450,0.310900,0.090383


In [316]:
from sklearn.model_selection import train_test_split

x = train.drop(['diagnosis', 'id'], axis=1)
y = train["diagnosis"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [317]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [318]:
# Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

sgd = SGDClassifier()
sgd.fit(x_train, y_train)
y_pred = sgd.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

sgd_acc = accuracy_score(y_test, y_pred)
print("Accuracy of SGD model is: ", sgd_acc)

Confusion Matrix: 
 [[48  3]
 [ 2 20]]


              precision    recall  f1-score   support

           B       0.96      0.94      0.95        51
           M       0.87      0.91      0.89        22

    accuracy                           0.93        73
   macro avg       0.91      0.93      0.92        73
weighted avg       0.93      0.93      0.93        73

Accuracy of SGD model is:  0.9315068493150684


# split 3

In [319]:
import pandas as pd

#import train and test CSV files
train = pd.read_csv("../5-fold_new/DATA/split3/train/train_split3.csv")
test = pd.read_csv("../5-fold_new/DATA/test.csv")

#take a look at the training data
train.describe(include="all")

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,3.640000e+02,364,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,...,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000
unique,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,3.592639e+07,NaN,14.092577,19.534615,91.627857,651.937363,0.095269,0.100669,0.084528,0.046416,...,16.115984,25.950742,106.059066,863.394505,0.130940,0.242281,0.259505,0.109807,0.282696,0.082433
std,1.401940e+08,NaN,3.524608,4.609915,24.274432,356.773643,0.013759,0.049271,0.077301,0.037252,...,4.723977,6.453569,32.663748,560.592855,0.021991,0.138997,0.195474,0.061740,0.053692,0.016335
min,8.670000e+03,NaN,7.691000,9.710000,47.920000,170.400000,0.052630,0.019380,0.000000,0.000000,...,8.678000,12.020000,54.490000,223.600000,0.071170,0.027290,0.000000,0.000000,0.156600,0.055250
25%,8.830692e+05,NaN,11.660000,16.330000,74.832500,414.500000,0.085402,0.062805,0.029400,0.020313,...,12.977500,21.150000,84.035000,509.425000,0.116200,0.145550,0.118475,0.065560,0.247250,0.070825
50%,9.141015e+05,NaN,13.375000,18.925000,86.290000,548.750000,0.094320,0.089065,0.057490,0.031905,...,14.905000,25.500000,97.260000,680.650000,0.131100,0.209900,0.207950,0.096305,0.280500,0.078710
75%,8.910808e+06,NaN,15.622500,22.080000,102.975000,759.275000,0.103825,0.127350,0.117250,0.065263,...,18.370000,29.757500,124.925000,1031.250000,0.142800,0.314150,0.363000,0.151650,0.310575,0.090145


In [320]:
from sklearn.model_selection import train_test_split

x = train.drop(['diagnosis', 'id'], axis=1)
y = train["diagnosis"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [321]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [322]:
# Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

sgd = SGDClassifier()
sgd.fit(x_train, y_train)
y_pred = sgd.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

sgd_acc = accuracy_score(y_test, y_pred)
print("Accuracy of SGD model is: ", sgd_acc)

Confusion Matrix: 
 [[46  1]
 [ 2 24]]


              precision    recall  f1-score   support

           B       0.96      0.98      0.97        47
           M       0.96      0.92      0.94        26

    accuracy                           0.96        73
   macro avg       0.96      0.95      0.95        73
weighted avg       0.96      0.96      0.96        73

Accuracy of SGD model is:  0.958904109589041


# split 4

In [323]:
import pandas as pd

#import train and test CSV files
train = pd.read_csv("../5-fold_new/DATA/split4/train/train_split4.csv")
test = pd.read_csv("../5-fold_new/DATA/test.csv")

#take a look at the training data
train.describe(include="all")

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,3.640000e+02,364,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,...,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000
unique,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2.780697e+07,NaN,14.179338,19.047637,92.187308,661.384615,0.095637,0.100962,0.085111,0.047447,...,16.212706,25.216181,106.648516,877.063736,0.130649,0.239348,0.254424,0.110032,0.285584,0.082188
std,1.147116e+08,NaN,3.634475,4.463224,24.955909,365.535254,0.014054,0.050086,0.078372,0.038701,...,4.931729,6.360105,33.988192,580.505111,0.022488,0.142427,0.195598,0.063964,0.058158,0.016447
min,8.670000e+03,NaN,7.691000,9.710000,47.920000,170.400000,0.052630,0.019380,0.000000,0.000000,...,8.678000,12.020000,54.490000,223.600000,0.071170,0.027290,0.000000,0.000000,0.156600,0.055210
25%,8.778670e+05,NaN,11.677500,15.827500,74.617500,418.325000,0.085218,0.063565,0.027045,0.019675,...,12.835000,20.522500,83.112500,501.300000,0.114300,0.138625,0.105875,0.063253,0.247950,0.070458
50%,9.130825e+05,NaN,13.290000,18.595000,85.585000,546.200000,0.094915,0.090020,0.058220,0.032770,...,14.730000,24.895000,96.665000,662.500000,0.129800,0.211700,0.209450,0.097330,0.280800,0.078650
75%,8.873089e+06,NaN,16.050000,21.620000,105.900000,789.675000,0.104650,0.128900,0.119575,0.066227,...,18.602500,28.735000,125.225000,1055.000000,0.142825,0.312250,0.363800,0.153125,0.313900,0.089862


In [324]:
from sklearn.model_selection import train_test_split

x = train.drop(['diagnosis', 'id'], axis=1)
y = train["diagnosis"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [325]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [326]:
# Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

sgd = SGDClassifier()
sgd.fit(x_train, y_train)
y_pred = sgd.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

sgd_acc = accuracy_score(y_test, y_pred)
print("Accuracy of SGD model is: ", sgd_acc)

Confusion Matrix: 
 [[47  0]
 [ 2 24]]


              precision    recall  f1-score   support

           B       0.96      1.00      0.98        47
           M       1.00      0.92      0.96        26

    accuracy                           0.97        73
   macro avg       0.98      0.96      0.97        73
weighted avg       0.97      0.97      0.97        73

Accuracy of SGD model is:  0.9726027397260274


# split 5

In [327]:
import pandas as pd

#import train and test CSV files
train = pd.read_csv("../5-fold_new/DATA/split5/train/train_split5.csv")
test = pd.read_csv("../5-fold_new/DATA/test.csv")

#take a look at the training data
train.describe(include="all")

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,3.640000e+02,364,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,...,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000
unique,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,3.999731e+07,NaN,14.137063,19.006923,91.838214,656.529396,0.094130,0.097540,0.082888,0.045802,...,16.189508,25.265220,106.466978,874.631319,0.129045,0.236734,0.253782,0.108538,0.284591,0.081397
std,1.539422e+08,NaN,3.529667,4.356193,24.292126,360.997435,0.013122,0.049086,0.078180,0.038304,...,4.830976,6.256582,33.208295,584.256393,0.021850,0.141072,0.195543,0.063720,0.056026,0.015782
min,8.670000e+03,NaN,8.219000,9.710000,53.270000,203.900000,0.062510,0.019380,0.000000,0.000000,...,8.952000,12.020000,56.650000,240.100000,0.071170,0.027290,0.000000,0.000000,0.160300,0.055210
25%,8.819442e+05,NaN,11.707500,15.917500,74.832500,420.175000,0.084495,0.060397,0.025983,0.019103,...,12.895000,20.537500,84.022500,510.275000,0.113925,0.135150,0.101288,0.060155,0.249700,0.070490
50%,9.050800e+05,NaN,13.200000,18.355000,85.250000,538.400000,0.093680,0.084995,0.055195,0.030640,...,14.680000,24.995000,96.180000,661.300000,0.129400,0.200400,0.207950,0.092015,0.281100,0.078095
75%,8.967931e+06,NaN,15.720000,21.532500,103.850000,768.325000,0.102825,0.122525,0.115350,0.065263,...,18.550000,28.772500,124.925000,1045.500000,0.141900,0.313175,0.371325,0.154450,0.311575,0.088173


In [328]:
from sklearn.model_selection import train_test_split

x = train.drop(['diagnosis', 'id'], axis=1)
y = train["diagnosis"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [329]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [330]:
# Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

sgd = SGDClassifier()
sgd.fit(x_train, y_train)
y_pred = sgd.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

sgd_acc = accuracy_score(y_test, y_pred)
print("Accuracy of SGD model is: ", sgd_acc)

Confusion Matrix: 
 [[49  1]
 [ 1 22]]


              precision    recall  f1-score   support

           B       0.98      0.98      0.98        50
           M       0.96      0.96      0.96        23

    accuracy                           0.97        73
   macro avg       0.97      0.97      0.97        73
weighted avg       0.97      0.97      0.97        73

Accuracy of SGD model is:  0.9726027397260274
